In [1]:
import re
import pandas as pd
import numpy as np
import sqlite3 
import sqlalchemy
import xlsxwriter

from sqlalchemy import create_engine
from pathlib import Path
from datetime import datetime, timedelta

In [2]:
import locale
locale.setlocale(locale.LC_ALL, '')

'Portuguese_Brazil.1252'

In [3]:
month = '8'
year  = '2019'

In [4]:
engine = create_engine("sqlite:///registros.db", echo=False)

In [5]:
pis_codes = engine.execute(f"SELECT DISTINCT pis FROM registros WHERE year = {year} AND month = {month}").fetchall()

print(len(pis_codes))

pis_codes

10


[('012303844632',),
 ('020071346877',),
 ('013118171501',),
 ('021065315459',),
 ('013979783277',),
 ('012699519512',),
 ('016002684248',),
 ('017048753097',),
 ('019004076622',),
 ('020637703280',)]

In [6]:
pis_code = pis_codes[4][0]

pis_code

'013979783277'

In [7]:
query = f"""SELECT * 
            FROM registros 
            WHERE year = {year} AND month = {month} AND pis = '{pis_code}'
        """

df = pd.read_sql(query, 
                 engine,
                 parse_dates='datetime')

df['date'] = df['datetime'].dt.date
df['rank'] = df.groupby(df.datetime.dt.day)['datetime'].rank().astype(int)

df.head(10)

,sequential_number,cod_op,day,month,year,hour,minutes,cod_input,pis,name,datetime,date,rank
0,39632,3,1,8,2019,7,56,None,013979783277,None,2019-08-01 07:56:00,2019-08-01,1
1,39639,3,1,8,2019,12,1,None,013979783277,None,2019-08-01 12:01:00,2019-08-01,2
2,39648,3,1,8,2019,13,10,None,013979783277,None,2019-08-01 13:10:00,2019-08-01,3
3,39654,3,1,8,2019,17,0,None,013979783277,None,2019-08-01 17:00:00,2019-08-01,4
4,39663,3,2,8,2019,8,19,None,013979783277,None,2019-08-02 08:19:00,2019-08-02,1
5,39668,3,2,8,2019,12,1,None,013979783277,None,2019-08-02 12:01:00,2019-08-02,2
6,39673,3,2,8,2019,12,58,None,013979783277,None,2019-08-02 12:58:00,2019-08-02,3
7,39683,3,2,8,2019,18,44,None,013979783277,None,2019-08-02 18:44:00,2019-08-02,4
8,39685,3,2,8,2019,19,1,None,013979783277,None,2019-08-02 19:01:00,2019-08-02,5
9,39687,3,2,8,2019,22,11,None,013979783277,None,2019-08-02 22:11:00,2019-08-02,6


In [8]:
table = df.pivot(index=['date'], columns='rank', values='datetime')

columns_reg = list(table.columns)

print(columns_reg)

table

[1, 2, 3, 4, 5, 6]


rank,1,2,3,4,5,6
date,,,,,,
2019-08-01,2019-08-01 07:56:00,2019-08-01 12:01:00,2019-08-01 13:10:00,2019-08-01 17:00:00,NaT,NaT
2019-08-02,2019-08-02 08:19:00,2019-08-02 12:01:00,2019-08-02 12:58:00,2019-08-02 18:44:00,2019-08-02 19:01:00,2019-08-02 22:11:00
2019-08-05,2019-08-05 08:00:00,2019-08-05 12:02:00,2019-08-05 13:10:00,2019-08-05 17:03:00,2019-08-05 19:38:00,2019-08-05 20:56:00
2019-08-06,2019-08-06 12:01:00,2019-08-06 13:26:00,2019-08-06 14:23:00,NaT,NaT,NaT
2019-08-08,2019-08-08 07:52:00,2019-08-08 12:01:00,2019-08-08 13:11:00,2019-08-08 17:00:00,NaT,NaT
2019-08-09,2019-08-09 07:55:00,2019-08-09 12:02:00,2019-08-09 13:09:00,2019-08-09 17:00:00,NaT,NaT
2019-08-12,2019-08-12 07:51:00,2019-08-12 12:00:00,2019-08-12 13:01:00,2019-08-12 17:01:00,2019-08-12 19:31:00,2019-08-12 21:07:00
2019-08-13,2019-08-13 07:57:00,2019-08-13 12:01:00,2019-08-13 13:12:00,2019-08-13 17:00:00,NaT,NaT
2019-08-14,2019-08-14 07:56:00,2019-08-14 12:00:00,2019-08-14 13:08:00,2019-08-14 17:01:00,NaT,NaT


In [14]:
tmp = df.groupby('date', as_index=True)['rank'].max()

print(tmp.name)

s1 = (tmp > 0) & (tmp <= 6) & (tmp % 2 == 0)
s1.name = 'is_valid'

rank


In [10]:
table2 = table.merge(s1, right_index=True, left_index=True)

table2

,1,2,3,4,5,6,is_valid
date,,,,,,,
2019-08-01,2019-08-01 07:56:00,2019-08-01 12:01:00,2019-08-01 13:10:00,2019-08-01 17:00:00,NaT,NaT,True
2019-08-02,2019-08-02 08:19:00,2019-08-02 12:01:00,2019-08-02 12:58:00,2019-08-02 18:44:00,2019-08-02 19:01:00,2019-08-02 22:11:00,True
2019-08-05,2019-08-05 08:00:00,2019-08-05 12:02:00,2019-08-05 13:10:00,2019-08-05 17:03:00,2019-08-05 19:38:00,2019-08-05 20:56:00,True
2019-08-06,2019-08-06 12:01:00,2019-08-06 13:26:00,2019-08-06 14:23:00,NaT,NaT,NaT,False
2019-08-08,2019-08-08 07:52:00,2019-08-08 12:01:00,2019-08-08 13:11:00,2019-08-08 17:00:00,NaT,NaT,True
2019-08-09,2019-08-09 07:55:00,2019-08-09 12:02:00,2019-08-09 13:09:00,2019-08-09 17:00:00,NaT,NaT,True
2019-08-12,2019-08-12 07:51:00,2019-08-12 12:00:00,2019-08-12 13:01:00,2019-08-12 17:01:00,2019-08-12 19:31:00,2019-08-12 21:07:00,True
2019-08-13,2019-08-13 07:57:00,2019-08-13 12:01:00,2019-08-13 13:12:00,2019-08-13 17:00:00,NaT,NaT,True
2019-08-14,2019-08-14 07:56:00,2019-08-14 12:00:00,2019-08-14 13:08:00,2019-08-14 17:01:00,NaT,NaT,True


In [11]:
table2['first_period']  = (table[2] - table[1]).fillna(pd.Timedelta(0))
table2['second_period'] = (table[4] - table[3]).fillna(pd.Timedelta(0))
table2['third_period']  = (table[6] - table[5]).fillna(pd.Timedelta(0))

table2['daily_work_hour'] = table2[['first_period', 'second_period', 'third_period']].sum(axis=1)

table2

,1,2,3,4,5,6,is_valid,first_period,second_period,third_period,daily_work_hour
date,,,,,,,,,,,
2019-08-01,2019-08-01 07:56:00,2019-08-01 12:01:00,2019-08-01 13:10:00,2019-08-01 17:00:00,NaT,NaT,True,0 days 04:05:00,0 days 03:50:00,0 days 00:00:00,0 days 07:55:00
2019-08-02,2019-08-02 08:19:00,2019-08-02 12:01:00,2019-08-02 12:58:00,2019-08-02 18:44:00,2019-08-02 19:01:00,2019-08-02 22:11:00,True,0 days 03:42:00,0 days 05:46:00,0 days 03:10:00,0 days 12:38:00
2019-08-05,2019-08-05 08:00:00,2019-08-05 12:02:00,2019-08-05 13:10:00,2019-08-05 17:03:00,2019-08-05 19:38:00,2019-08-05 20:56:00,True,0 days 04:02:00,0 days 03:53:00,0 days 01:18:00,0 days 09:13:00
2019-08-06,2019-08-06 12:01:00,2019-08-06 13:26:00,2019-08-06 14:23:00,NaT,NaT,NaT,False,0 days 01:25:00,0 days 00:00:00,0 days 00:00:00,0 days 01:25:00
2019-08-08,2019-08-08 07:52:00,2019-08-08 12:01:00,2019-08-08 13:11:00,2019-08-08 17:00:00,NaT,NaT,True,0 days 04:09:00,0 days 03:49:00,0 days 00:00:00,0 days 07:58:00
2019-08-09,2019-08-09 07:55:00,2019-08-09 12:02:00,2019-08-09 13:09:00,2019-08-09 17:00:00,NaT,NaT,True,0 days 04:07:00,0 days 03:51:00,0 days 00:00:00,0 days 07:58:00
2019-08-12,2019-08-12 07:51:00,2019-08-12 12:00:00,2019-08-12 13:01:00,2019-08-12 17:01:00,2019-08-12 19:31:00,2019-08-12 21:07:00,True,0 days 04:09:00,0 days 04:00:00,0 days 01:36:00,0 days 09:45:00
2019-08-13,2019-08-13 07:57:00,2019-08-13 12:01:00,2019-08-13 13:12:00,2019-08-13 17:00:00,NaT,NaT,True,0 days 04:04:00,0 days 03:48:00,0 days 00:00:00,0 days 07:52:00
2019-08-14,2019-08-14 07:56:00,2019-08-14 12:00:00,2019-08-14 13:08:00,2019-08-14 17:01:00,NaT,NaT,True,0 days 04:04:00,0 days 03:53:00,0 days 00:00:00,0 days 07:57:00


In [13]:
table2.columns

Index([                1,                 2,                 3,
                       4,                 5,                 6,
              'is_valid',    'first_period',   'second_period',
          'third_period', 'daily_work_hour'],
      dtype='object')

In [12]:
index = pd.date_range(start='2019-08-01', end='2019-08-31')

index.day_name()

table3 = pd.DataFrame(index.day_name(), index=index, columns=['WeekName'])

table4 = table3.copy()

table3

,WeekName
2019-08-01,Thursday
2019-08-02,Friday
2019-08-03,Saturday
2019-08-04,Sunday
2019-08-05,Monday
2019-08-06,Tuesday
2019-08-07,Wednesday
2019-08-08,Thursday
2019-08-09,Friday
2019-08-10,Saturday


In [13]:
table3['first_period']  = (table[2] - table[1]).fillna(pd.Timedelta(0))
table3['second_period'] = (table[4] - table[3]).fillna(pd.Timedelta(0))
table3['third_period']  = (table[6] - table[5]).fillna(pd.Timedelta(0))

# table3.fillna(pd.Timedelta(0), inplace=True)

table3['daily_work_hour'] = table3[['first_period', 'second_period', 'third_period']].sum(axis=1)

In [14]:
table3

,WeekName,first_period,second_period,third_period,daily_work_hour
2019-08-01,Thursday,0 days 04:05:00,0 days 03:50:00,0 days 00:00:00,0 days 07:55:00
2019-08-02,Friday,0 days 03:42:00,0 days 05:46:00,0 days 03:10:00,0 days 12:38:00
2019-08-03,Saturday,NaT,NaT,NaT,0 days 00:00:00
2019-08-04,Sunday,NaT,NaT,NaT,0 days 00:00:00
2019-08-05,Monday,0 days 04:02:00,0 days 03:53:00,0 days 01:18:00,0 days 09:13:00
2019-08-06,Tuesday,0 days 01:25:00,0 days 00:00:00,0 days 00:00:00,0 days 01:25:00
2019-08-07,Wednesday,NaT,NaT,NaT,0 days 00:00:00
2019-08-08,Thursday,0 days 04:09:00,0 days 03:49:00,0 days 00:00:00,0 days 07:58:00
2019-08-09,Friday,0 days 04:07:00,0 days 03:51:00,0 days 00:00:00,0 days 07:58:00
2019-08-10,Saturday,NaT,NaT,NaT,0 days 00:00:00


In [ ]:
columns = [
    'WeekName',
     1, 2, 3, 4, 5, 6,
    'first_period', 
    'second_period', 
    'third_period'
    'daily_work_hour'
]

In [15]:
table4['first_period']  = (table[2] - table[1]).fillna(pd.Timedelta(0)).dt.total_seconds()
table4['second_period'] = (table[4] - table[3]).fillna(pd.Timedelta(0)).dt.total_seconds()
table4['third_period']  = (table[6] - table[5]).fillna(pd.Timedelta(0)).dt.total_seconds()

table4.fillna(0.0, inplace=True)

In [16]:
table4['daily_work_seconds'] = table4[['first_period', 'second_period', 'third_period']].sum(axis=1)

In [17]:
s5 = table4['daily_work_seconds'] - pd.Timedelta(hours=8).total_seconds()

table4 ['signal'] = s5 >= 0

table4['diff'] = s5

table4['diff_abs'] = np.abs(s5)

In [18]:
table4

,WeekName,first_period,second_period,third_period,daily_work_seconds,signal,diff,diff_abs
2019-08-01,Thursday,14700.0,13800.0,0.0,28500.0,False,-300.0,300.0
2019-08-02,Friday,13320.0,20760.0,11400.0,45480.0,True,16680.0,16680.0
2019-08-03,Saturday,0.0,0.0,0.0,0.0,False,-28800.0,28800.0
2019-08-04,Sunday,0.0,0.0,0.0,0.0,False,-28800.0,28800.0
2019-08-05,Monday,14520.0,13980.0,4680.0,33180.0,True,4380.0,4380.0
2019-08-06,Tuesday,5100.0,0.0,0.0,5100.0,False,-23700.0,23700.0
2019-08-07,Wednesday,0.0,0.0,0.0,0.0,False,-28800.0,28800.0
2019-08-08,Thursday,14940.0,13740.0,0.0,28680.0,False,-120.0,120.0
2019-08-09,Friday,14820.0,13860.0,0.0,28680.0,False,-120.0,120.0
2019-08-10,Saturday,0.0,0.0,0.0,0.0,False,-28800.0,28800.0


In [19]:
table4.groupby('signal')['diff'].sum()

signal
False   -438540.0
True      38880.0
Name: diff, dtype: float64

In [20]:
table3['diff_abs'] = pd.to_timedelta(table4['diff_abs'], unit='sec')

table3['signal'] = table4['signal']

In [22]:
table3['diff_abs'] = table3['diff_abs'].mask(table3['WeekName'].isin(['Saturday', 'Sunday']), pd.Timedelta(0))

In [23]:
cols = ['first_period', 'second_period', 'third_period', 'daily_work_seconds', 'diff']

In [24]:
table3.groupby('signal')['diff_abs'].sum()

signal
False   2 days 01:49:00
True    0 days 10:48:00
Name: diff_abs, dtype: timedelta64[ns]

In [25]:
table3[['first_period', 'second_period', 'third_period']].isna().sum(axis=1)

2019-08-01    0
2019-08-02    0
2019-08-03    3
2019-08-04    3
2019-08-05    0
2019-08-06    0
2019-08-07    3
2019-08-08    0
2019-08-09    0
2019-08-10    3
2019-08-11    3
2019-08-12    0
2019-08-13    0
2019-08-14    0
2019-08-15    0
2019-08-16    0
2019-08-17    3
2019-08-18    3
2019-08-19    3
2019-08-20    3
2019-08-21    0
2019-08-22    0
2019-08-23    0
2019-08-24    3
2019-08-25    3
2019-08-26    0
2019-08-27    0
2019-08-28    0
2019-08-29    0
2019-08-30    0
2019-08-31    3
Freq: D, dtype: int64